In [34]:
import os
import igraph as ig
from igraph import Graph

import pickle
import random
import pandas as pd
import numpy as np

from sklearn.metrics import normalized_mutual_info_score
import community as community_louvain
import networkx as nx

#https://igraph.org/python/doc/api/igraph.Graph.html

In [ ]:
# Load community belonging
# Iterate resulting graphs
# Load graph
# Run community detection algos
# Save results

In [45]:
dataset_name = 'dancer_01'
graphs_folder = 'D:/Pablo/clases/UJM/2. Semester, 2021/Mining Uncertain Social Networks/Experiments/datasets/dancer_01/resulting_graphs/'
output_folder = 'D:/Pablo/clases/UJM/2. Semester, 2021/Mining Uncertain Social Networks/Experiments/datasets/dancer_01/'
community_file = 'D:/Pablo/clases/UJM/2. Semester, 2021/Mining Uncertain Social Networks/Experiments/datasets/dancer_01/dancer_01_comm.txt'

In [36]:
# Load community belonging
communities = {}
with open(community_file) as f:
    for line in f:
       (key, val) = line.split()
       communities[key] = val

In [56]:
# Unweighted analysis

results_rows = []

# Value to be used during all of the runs
p_size = len(set(communities.values()))

# Iterate generated graphs
for file_name in os.listdir(graphs_folder+'add_edges/'):
    # Load graph
    graph = Graph.Read_Ncol(graphs_folder+'add_edges/'+file_name, directed=False)
    
    # Add community belonging
    for vertex in graph.vs:
        vertex['community'] = communities[vertex['name']]

    # louvian
    method = 'Louvain-igraph'

    louvian = ig.Graph.community_multilevel(graph, return_levels=True)
    louvian = louvian[len(louvian)-1]
    p_louvian = len(set(louvian.membership))

    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P',
                 'value':p_louvian})    
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P*/P',
                 'value':p_size/p_louvian})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'Modularity',
                 'value':graph.modularity(louvian.membership)})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'NMI',
                 'value':normalized_mutual_info_score(graph.vs['community'], louvian.membership)})

    # FastGreedy
    method = 'Fastgreedy'

    fg = ig.Graph.community_fastgreedy(graph)
    p_fg = fg.optimal_count
    fg = fg.as_clustering()
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P',
                 'value':p_fg})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P*/P',
                 'value':p_size/p_fg})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'Modularity',
                 'value':graph.modularity(fg.membership)})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'NMI',
                 'value':normalized_mutual_info_score(graph.vs['community'], fg.membership)})
    # Infomap
    method = 'Infomap'

    infomap = ig.Graph.community_infomap(graph)
    p_im = len(set(infomap.membership))
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P',
                 'value':p_im})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P*/P',
                 'value':p_size/p_im})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'Modularity',
                 'value':graph.modularity(infomap.membership)})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'NMI',
                 'value':normalized_mutual_info_score(graph.vs['community'], infomap.membership)})

    # Label Propagation
    method = 'Label Propagation'

    lp = ig.Graph.community_label_propagation(graph)
    p_lp = len(set(lp.membership))
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P',
                 'value':p_lp})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P*/P',
                 'value':p_size/p_lp})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'Modularity',
                 'value':graph.modularity(lp.membership)})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'NMI',
                 'value':normalized_mutual_info_score(graph.vs['community'], lp.membership)})

    # Louvain
    method = 'Louvain'

    # Creating nx Graph to for other Louvain implementation
    nxG = nx.Graph()
    nxG.add_nodes_from([vertex.index for vertex in graph.vs])
    nxG.add_edges_from([edge.tuple for edge in graph.es])

    lv_partition = community_louvain.best_partition(nxG)
    p_lv = len(set(lv_partition.values()))
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P',
                 'value':p_lv})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P*/P',
                 'value':p_size/p_lv})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'Modularity',
                 'value':graph.modularity(lv_partition.values())})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'NMI',
                 'value':normalized_mutual_info_score(graph.vs['community'], pd.Series(lv_partition.values()))})

community_algos = pd.DataFrame(results_rows)

community_algos.to_csv(output_folder+'add_edges_communities.csv', index=False)

In [55]:
# Weighted new edges analysis

results_rows = []

# Value to be used during all of the runs
p_size = len(set(communities.values()))

# Iterate generated graphs
for file_name in os.listdir(graphs_folder+'add_weighted/'):
    # Load graph
    graph = Graph.Read_Ncol(graphs_folder+'add_weighted/'+file_name, directed=False)
    
    # Add community belonging
    for vertex in graph.vs:
        vertex['community'] = communities[vertex['name']]

    # louvian
    method = 'Louvain-igraph'

    louvian = ig.Graph.community_multilevel(graph, weights='weight', return_levels=True)
    louvian = louvian[len(louvian)-1]
    p_louvian = len(set(louvian.membership))

    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P',
                 'value':p_louvian})    
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P*/P',
                 'value':p_size/p_louvian})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'Modularity',
                 'value':graph.modularity(louvian.membership)})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'NMI',
                 'value':normalized_mutual_info_score(graph.vs['community'], louvian.membership)})

    # FastGreedy
    method = 'Fastgreedy'

    fg = ig.Graph.community_fastgreedy(graph, weights='weight')
    p_fg = fg.optimal_count
    fg = fg.as_clustering()
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P',
                 'value':p_fg})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P*/P',
                 'value':p_size/p_fg})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'Modularity',
                 'value':graph.modularity(fg.membership)})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'NMI',
                 'value':normalized_mutual_info_score(graph.vs['community'], fg.membership)})
    # Infomap
    method = 'Infomap'

    infomap = ig.Graph.community_infomap(graph, edge_weights='weight')
    p_im = len(set(infomap.membership))
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P',
                 'value':p_im})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P*/P',
                 'value':p_size/p_im})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'Modularity',
                 'value':graph.modularity(infomap.membership)})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'NMI',
                 'value':normalized_mutual_info_score(graph.vs['community'], infomap.membership)})

    # Label Propagation
    method = 'Label Propagation'

    lp = ig.Graph.community_label_propagation(graph, weights='weight')
    p_lp = len(set(lp.membership))
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P',
                 'value':p_lp})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P*/P',
                 'value':p_size/p_lp})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'Modularity',
                 'value':graph.modularity(lp.membership)})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'NMI',
                 'value':normalized_mutual_info_score(graph.vs['community'], lp.membership)})

    # Louvain
    method = 'Louvain'

    # Creating nx Graph to for other Louvain implementation
    nxG = nx.Graph()
    nxG.add_nodes_from([vertex.index for vertex in graph.vs])
    nxG.add_edges_from([(edge.tuple[0],edge.tuple[1],{'weight':edge['weight']}) for edge in graph.es])

    lv_partition = community_louvain.best_partition(nxG, weight='weight')
    p_lv = len(set(lv_partition.values()))
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P',
                 'value':p_lv})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'P*/P',
                 'value':p_size/p_lv})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'Modularity',
                 'value':graph.modularity(lv_partition.values())})
    results_rows.append({'dataset':dataset_name, 'file_name':file_name, 'method':method, 'metric':'NMI',
                 'value':normalized_mutual_info_score(graph.vs['community'], pd.Series(lv_partition.values()))})

community_algos = pd.DataFrame(results_rows)
community_algos.to_csv(output_folder+'add_weighted_communities.csv', index=False)